In [2]:
import pandas as pd
import geopandas as gpd
import requests
import plotly.express as px
import json
from shapely.geometry import shape
from dataclasses import dataclass, field

In [44]:
@dataclass
class BaseRequest:
    """base request for this API"""

    base_url="https://services2.arcgis.com/HdTo6HJqh92wn4D8/arcgis/rest/services/Building_Permit_Applications_Feature_Layer_view/FeatureServer/0/query"
    params:dict = field(init=True, default_factory=lambda : {"outFields":"*", "where": "1=1","f":"json"})

    # post-initalized data
    data: pd.DataFrame = field(init=False)

    def __post_init__(self): 
        """post initialized build of data"""
        self.data = self.request()

    def request(self) -> pd.DataFrame: 
        """make request of class"""
        response = requests.get(self.base_url, params=self.params)

        if not response.status_code == 200: 
            raise ValueError(f"{response.reason}")

        request_json = response.json()
        data=pd.json_normalize(
            request_json,
            record_path=["features"]
        )
        
        data.columns = data.columns.str.replace("attributes.","")\
            .str.lower()\
            .str.removesuffix("__")\
            .str.replace(".","_")
        
        return data

In [48]:
@dataclass
class PermitsApplied(BaseRequest): 
    """permits applied for"""
    base_url="https://services2.arcgis.com/HdTo6HJqh92wn4D8/arcgis/rest/services/Building_Permit_Applications_Feature_Layer_view/FeatureServer/0/query"

@dataclass
class PermitsIssued(BaseRequest): 
    """permits applied for"""
    base_url="https://services2.arcgis.com/HdTo6HJqh92wn4D8/arcgis/rest/services/Building_Permits_Issued_2/FeatureServer/0/query"

applied = PermitsApplied()
issued = PermitsIssued()

In [ ]:
print(response.json().keys())
data=pd.json_normalize(
    response.json(),
    record_path=["features"]
)
data.columns = data.columns.str.replace("attributes.","")\
    .str.lower()\
    .str.removesuffix("__")\
    .str.replace(".","_")


data.head(3)

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'exceededTransferLimit', 'features'])


,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,per_subty,ivr_trk_,purpose,council_dist,lon,lat,objectid,zip,geometry_x,geometry_y
0,2023023374,Building Residential - Addition,Single Family Residence,10412024600,1681189200000,None,0,1916 18TH AVE S,NASHVILLE,TN,...,CAA01R301,4302991,Addition and outbuilding construction must ...,18,-86.796767,36.134014,1,37212,-9.662172e+06,4.319077e+06
1,D2020047607,Building Use & Occupancy,"Accessory Structure, Pools - Residential",11704002200,1698123600000,None,100000,2509 BELMONT BLVD,NASHVILLE,TN,...,CAA14U017,3856896,Proposed 13' x 32' Swimming Pool in rear yard.,18,-86.794539,36.125197,2,37212,-9.661924e+06,4.317862e+06
2,D2020068970,Building Residential - Addition,Single Family Residence,06308004200,1675317600000,None,105,4004 UNIVERSITY AVE,OLD HICKORY,TN,...,CAA01R301,3891358,"adding a 6' wide by 17.5' deep, single story a...",11,-86.633246,36.228955,3,37138,-9.643969e+06,4.332171e+06


In [3]:

# Function to fetch and process the GeoJSON data
def fetch_geojson_data(url):
    """
    Fetches GeoJSON data from the provided URL and converts it to a GeoDataFrame
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        geojson_data = response.json()
        
        # Extract features and properties
        features = []
        for feature in geojson_data['features']:
            properties = feature['properties']
            geometry = shape(feature['geometry'])
            features.append({
                'geometry': geometry,
                **properties
            })
        
        # Create GeoDataFrame
        gdf = gpd.GeoDataFrame(features)
        return gdf
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None



In [4]:
# Fetch and process the data
url = "https://services2.arcgis.com/HdTo6HJqh92wn4D8/arcgis/rest/services/Pavement_view/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson"
gdf = fetch_geojson_data(url)


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:

# Basic data cleaning and preprocessing
def preprocess_data(gdf):
    """
    Clean and preprocess the GeoDataFrame
    """
    if gdf is None:
        return None
    
    # Remove any duplicate records
    gdf = gdf.drop_duplicates()
    
    # Convert geometry to WGS84 (EPSG:4326) if needed
    if gdf.crs is None or gdf.crs != 'EPSG:4326':
        gdf = gdf.to_crs('EPSG:4326')
    
    # Extract coordinates for plotting
    gdf['longitude'] = gdf.geometry.centroid.x
    gdf['latitude'] = gdf.geometry.centroid.y
    
    return gdf

# Preprocess the data
processed_gdf = preprocess_data(gdf)



In [ ]:
# Create visualization functions
def plot_points_map(gdf, color_column=None, title="Pavement Data Visualization"):
    """
    Create an interactive map using Plotly
    """
    if gdf is None:
        return None
    
    fig = px.scatter_mapbox(
        gdf,
        lat='latitude',
        lon='longitude',
        color=color_column if color_column else None,
        hover_data=gdf.columns,
        zoom=12,
        title=title
    )
    
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={"r":0,"t":30,"l":0,"b":0}
    )
    
    return fig



In [ ]:
def create_choropleth(gdf, color_column, title="Choropleth Map"):
    """
    Create a choropleth map using Plotly
    """
    if gdf is None:
        return None
    
    fig = px.choropleth_mapbox(
        gdf,
        geojson=gdf.geometry.__geo_interface__,
        locations=gdf.index,
        color=color_column,
        center={"lat": gdf.latitude.mean(), "lon": gdf.longitude.mean()},
        zoom=12,
        title=title
    )
    
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={"r":0,"t":30,"l":0,"b":0}
    )
    
    return fig


In [ ]:

# Create example visualizations
point_map = plot_points_map(processed_gdf)
# Assuming there's a numeric column called 'CONDITION' in your data
if 'CONDITION' in processed_gdf.columns:
    choropleth = create_choropleth(processed_gdf, 'CONDITION', "Pavement Condition Map")